# List Dtypes

In [1]:
import polars as pl

In [3]:
df_lists = pl.DataFrame({
    "ints": [[0, 1], [2, 3]],
    "floats": [[0.0, 1], [2, 3]],
    "strings": [["0", "1"], ["2", "3"]]
}, strict=False)
df_lists

ints,floats,strings
list[i64],list[f64],list[str]
"[0, 1]","[0.0, 1.0]","[""0"", ""1""]"
"[2, 3]","[2.0, 3.0]","[""2"", ""3""]"


In [4]:
df_lists[0, "ints"]

""
i64
0
1


In [5]:
pl.Config.set_fmt_table_cell_list_len(20)

polars.config.Config

In [7]:
(
    df_lists
    .select(
        pl.col(pl.List(pl.Int64))
    )
)

ints
list[i64]
"[0, 1]"
"[2, 3]"


In [8]:
(
    pl.DataFrame(
        {
            "values": [
                [0, 1],
                [2, 3, 4],
                [4, 5, 6, 7, 8]
            ]
        }
    )
)

values
list[i64]
"[0, 1]"
"[2, 3, 4]"
"[4, 5, 6, 7, 8]"


In [9]:
(
    df_lists
    .with_columns(
        ints_array = pl.col("ints").cast(pl.Array(width=2, inner=pl.Int64))
    )
    .select("ints", "ints_array")
)

C:\Users\yk279\AppData\Local\Temp\ipykernel_12104\2374781562.py:4: DeprecationWarning: the `width` parameter for `Array` is deprecated. Use `shape` instead.
(Deprecated in version 0.20.31)
  ints_array = pl.col("ints").cast(pl.Array(width=2, inner=pl.Int64))


ints,ints_array
list[i64],"array[i64, 2]"
"[0, 1]","[0, 1]"
"[2, 3]","[2, 3]"


In [ ]:
#explode